# W2D1 Tutorial 5: Estimate model parameters - maximum likelihood, Moments, Bayesian

#### __Week 2, Day 4, Extremes & Vulnerability__
##### __Content creators:__ Matthias Aengenheyster, Joeri Reinders
##### __Content reviewers:__ TBD
##### __Content editors:__ TBD
##### __Production editors:__ TBD
##### __Our 2023 Sponsors:__ TBD

## Tutorial Objectives:

In the previous tutorials we have fitted GEV distributions to our observational data.

For this next tutorial we will dive  more into how this fitting is performed, that is how the model parameters are estimated. 
As you might remember from the previous tutorials the GEV distribution has three variables: the location,
scale, and shape parameter. By default the ef.fit_return_levels_sdfc function uses a 'maximum likelihood estimation' (MLE).

A MLE is an estimator, a function of your data that gives you an approximation of the parameters. In a 
MLE we compute the likelihood that a given set of parameters would return the observed data. The likelihood
is given by the earlier discussed probability density function that results from a set of parameters. 

We thus made two very important assumptions in our previous tutorials:
1. We assumed we were working with the GEV distribution
2. We assumed the MLE method would give us the best parameter estimation. 

In this tutorial we will test these assumptions by using different parameter estimation
methods to see if our parameter values signficiantly change. Next we show you how you can use different 
models.

By the end of the tutorial, you will be able to:
1. Explain different parameter estimation methods
2. Compute the different parameter values fitted by different parameter estimation methods.

## Setup

In [3]:
# Installs

In [4]:
# !pip install -q condacolab
# import condacolab
# condacolab.install()
# #install dependencies - taken from <Yosmely Bermúdez> comments for Tutorial 6
# # We need this to install eigen which is needed for SDFC to install correctly
# !mamba install eigen numpy matplotlib seaborn pandas cartopy scipy texttable intake xarrayutils xmip cf_xarray intake-esm
# !pip install -v https://github.com/yrobink/SDFC/archive/master.zip#subdirectory=python
# !pip install https://github.com/njleach/mystatsfunctions/archive/master.zip

## Imports and function definitions

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats

import extremes_functions as ef
from mystatsfunctions import OLSE,LMoments
import SDFC as sd

In [6]:
def estimate_return_level(quantile,model):
    loc, scale, shape = model.coef_
    level = loc - scale / shape * (1 - (-np.log(quantile))**(-shape))
    # level = stats.genextreme.ppf(quantile,-shape,loc=loc,scale=scale)
    return level

We start by opening again the precipitation data from Germany once more, and fit a GEV distribution to it using
the MLE method, you can do this by adding "method = MLE" to the fit function. Have a look at the parameter
values.

In [7]:
import os, pooch

fname = 'precipitationGermany_1920-2022.csv'
if not os.path.exists(fname):
    url = "https://osf.io/xs7h6/download"
    fname = pooch.retrieve(url, known_hash=None)

data = pd.read_csv(fname, index_col=0).set_index('years')

data.columns=['precipitation']
precipitation = data.precipitation

In [8]:
fit, model = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='mle')

In [9]:
model

+-----------+--------+------------+--------+----------------+----------------+
| GEV (mle) |  Link  |    Type    |  coef  | Quantile 0.025 | Quantile 0.975 |
+===========+========+============+========+================+================+
| loc       | IdLink | Stationary | 26.354 | 25.222         | 27.521         |
+-----------+--------+------------+--------+----------------+----------------+
| scale     | IdLink | Stationary | 7.369  | 6.522          | 8.866          |
+-----------+--------+------------+--------+----------------+----------------+
| shape     | IdLink | Stationary | 0.047  | -0.045         | 0.149          |
+-----------+--------+------------+--------+----------------+----------------+

There are two other parameter estimation methods that we will explore and compare with the MLE: the
L-moments method, and a Baseyian method. The L-moments describe the shape of a probability distribution,
just like regular moments, and are a linear combination of the order statistics (L stands for linear).
The GEV-distribution parameters can be computed through a set of equations based from these L-moments.

The Bayesian technique on the other hand, [explain]

Estimate the GEV parameters based on these two method by changing the 'method' to either "Lmoments" and 
"Bayesian". After that compute the 100-year flood for all three parameter sets (MLE, Lmoments and Bayesian)

In [10]:
fit_moments, model_moments = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='moments')

In [11]:
fit_moments

<xarray.Dataset>
Dimensions:            (return period: 999, N: 10, return_period_obs: 103)
Coordinates:
  * return period      (return period) float64 1.1 2.1 3.1 ... 997.1 998.1 999.1
  * N                  (N) int64 0 1 2 3 4 5 6 7 8 9
  * return_period_obs  (return_period_obs) float64 1.01 1.02 1.03 ... 52.0 104.0
Data variables:
    return level       (return period, N) float64 24.33 26.14 ... 41.73 40.11
    mu                 (N) float64 26.15 27.85 26.59 26.61 ... 28.34 27.04 25.73
    sigma              (N) float64 2.08 1.953 1.995 2.002 ... 1.943 2.128 2.083
    xi                 (N) float64 1e-08 1e-08 1e-08 1e-08 ... 1e-08 1e-08 1e-08
    return_level_obs   (return_period_obs) float64 15.6 16.2 16.9 ... 59.9 69.5

In [12]:
model_moments

+---------------+--------+------------+--------+----------------+----------------+
| GEV (moments) |  Link  |    Type    |  coef  | Quantile 0.025 | Quantile 0.975 |
+===============+========+============+========+================+================+
| loc           | IdLink | Stationary | 26.526 | 25.824         | 28.227         |
+---------------+--------+------------+--------+----------------+----------------+
| scale         | IdLink | Stationary | 2.042  | 1.945          | 2.151          |
+---------------+--------+------------+--------+----------------+----------------+
| shape         | IdLink | Stationary | 0      | 0              | 0              |
+---------------+--------+------------+--------+----------------+----------------+

In [13]:
prior = stats.multivariate_normal(mean= model.coef_, cov = np.cov(model.coefs_bootstrap.T), allow_singular=True)

In [14]:
fit_bayes, model_bayes = ef.fit_return_levels_sdfc(precipitation.values,times=np.arange(1.1,1000),periods_per_year=1,kind='GEV',N_boot=10,full=True,model=True,method='bayesian',prior=prior,mcmc_init=model.coef_)

/home/a/aengenheyster/nobackups/bin/conda/envs/science39/lib/python3.9/site-packages/SDFC-0.5.0-py3.9-linux-x86_64.egg/SDFC/__AbstractLaw.py:438: RuntimeWarning: invalid value encountered in double_scalars
  p_accept = np.exp( p_next - p_current )


In [15]:
fit_bayes

<xarray.Dataset>
Dimensions:            (return period: 999, N: 10, return_period_obs: 103)
Coordinates:
  * return period      (return period) float64 1.1 2.1 3.1 ... 997.1 998.1 999.1
  * N                  (N) int64 0 1 2 3 4 5 6 7 8 9
  * return_period_obs  (return_period_obs) float64 1.01 1.02 1.03 ... 52.0 104.0
Data variables:
    return level       (return period, N) float64 -10.28 16.56 ... -54.93 0.7239
    mu                 (N) float64 2.848 22.91 -0.2313 ... 19.8 -4.837 -1.897
    sigma              (N) float64 0.7321 7.667 -5.013 ... 9.12 -1.118 1.219
    xi                 (N) float64 -5.197 0.1231 3.704 ... 0.4384 -0.4435
    return_level_obs   (return_period_obs) float64 15.6 16.2 16.9 ... 59.9 69.5

In [16]:
model_bayes

+----------------+--------+------------+--------+----------------+----------------+
| GEV (bayesian) |  Link  |    Type    |  coef  | Quantile 0.025 | Quantile 0.975 |
+================+========+============+========+================+================+
| loc            | IdLink | Stationary | 26.296 | -4.711         | 22.791         |
+----------------+--------+------------+--------+----------------+----------------+
| scale          | IdLink | Stationary | 7.399  | -4.334         | 8.951          |
+----------------+--------+------------+--------+----------------+----------------+
| shape          | IdLink | Stationary | 0.053  | -4.127         | 3.365          |
+----------------+--------+------------+--------+----------------+----------------+

In [17]:
period = 100
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

MLE: 64.21
Moments: 35.92
Bayes: 64.80


In [14]:
period = 50
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

MLE: 57.92
Moments: 34.49
Bayes: 58.73


In [15]:
period = 500
quantile = 1-1/period

print('MLE: %.2f' % estimate_return_level(quantile,model))
print('Moments: %.2f' % estimate_return_level(quantile,model_moments))
print('Bayes: %.2f' % estimate_return_level(quantile,model_bayes))

MLE: 79.55
Moments: 39.21
Bayes: 81.44


In [16]:
gev = LMoments.gev()
gev.fit(precipitation.values)
gev.X, gev.a, gev.k

(26.453249703966794, 7.687907961826535, -0.01055113362649465)

## Exercise: What can you say about the parameters estimated by the different methods? 